In [33]:
import json
from IPython.display import display, Markdown
import tiktoken
from cadence.agents.reasoning import ReasoningAgent

In [34]:
! python tools/collect_code.py --root cadence --ext .py --out agent_context/code.json

Wrote 22 files → agent_context/code.json


In [35]:
! python tools/collect_code.py --root docs --ext .md .json .mermaid .py --out agent_context/docs.json

Wrote 4 files → agent_context/docs.json


In [ ]:


with open("agent_context/code.json", "r") as f:
    codebase = json.load(f)
with open("agent_context/docs.json", "r") as f:
    docs = json.load(f)
with open("agent_context/module_contexts.json", "r") as f:
    contexts = json.load(f)
with open("backlog/backlog.json", "r") as f:
    backlog = json.load(f)

system_prompt = f"""
You are an AI-enhanced co-developer and co-designer along with your human counterpart.  Your role is to optimize the project by mining the best, most creative ideas from him and leveraging each of them into a high-quality, high-powered, autonomous, multi-agent development platform. You balance ambitious ideas with practical progress.
You have access to the docs, a JSON data structure that includes context on each module, the codebase itself, and a backlog of task to-dos.
=================
Project Docs:
{docs}
=================
Module Contexts (agent-related):
{contexts}
=================
Actual Code (agent-related):
{codebase}
=================
Backlog:
{backlog}
"""


enc = tiktoken.get_encoding("o200k_base")
tokens = enc.encode(system_prompt)
len(tokens)

24341

In [ ]:

agent = ReasoningAgent()
agent.reset_context(system_prompt)
agent.messages = agent.messages[:1]
agent.messages

[{'role': 'system',
  'content': '\nYou are an AI-enhanced co-developer and co-designer along with your human counterpart.  Your role is to optimize the project by mining the best, most creative ideas from him and leveraging each of them into a high-quality, high-powered, autonomous, multi-agent development platform. You balance ambitious ideas with practical progress.\nYou have access to the docs, a JSON data structure that includes context on each module, the codebase itself, and a backlog of task to-dos.\n=================\nProject Docs:\n{\'docs/DEV_PROCESS.md\': \'\\n# CADENCE DEVELOPMENT\\xa0PROCESS\\n\\n*Last‑updated: 2025‑06‑20*\\n\\n## 1\\xa0·\\xa0Overview\\n\\nOne canonical document defines *what* must happen and *in what order*. All other docs reference this file to avoid drift.\\n\\n## 2\\xa0·\\xa0Core Workflow Phases\\n\\n| Phase        | Role\\xa0(Class)     | Critical\\xa0Interfaces                           | Fail\\xa0Criterion                 |\\n| ------------ | -----

In [38]:
response = agent.run_interaction(
"""
Let's get started. Put togetehr some clear instructions for what to do next.
"""
)
display(Markdown(response))

[2025-06-20 15:58:04,277] INFO LLM sync call: model=o3-2025-04-16, msgs_len=2


🧭 Immediate Next Steps — 45-Minute Kick-off Checklist  
(aim: reach “red” state with a failing test that the autonomous loop can later turn green)

1. Workspace & Environment
   • Create / activate a fresh Python ≥3.10 virtual-env.  
   • `pip install -U openai python-dotenv pytest tabulate streamlit pandas`  
   • Ensure `OPENAI_API_KEY` is exported (or present in `.env`).

2. Baseline Commit
   • `git init` (if not already) → commit current code as “baseline-0”.

3. Seed the Toy Defect (Story S-MVP-1.1)
   a. `mkdir -p cadence/utils`  
   b. `cadence/utils/add.py`  
      ```python
      def add(x: int, y: int) -> int:
          """Intentionally wrong implementation for MVP red→green demo."""
          return x - 1 + y
      ```
   c. `tests/test_add.py`  
      ```python
      from cadence.utils.add import add

      def test_add():
          assert add(2, 3) == 5
      ```
   d. `pytest` should now FAIL. Commit as “seed-defect”.

4. Backlog File
   • `dev_backlog.json` → paste the single micro-task that will drive the loop:

   ```json
   [
     {
       "id": "bug-fix-add-001",
       "title": "Fix utils.add.add() so that add(2,3)==5",
       "type": "micro",
       "status": "open",
       "created_at": "2025-06-21T00:00:00Z",
       "diff": {
         "file": "cadence/utils/add.py",
         "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
         "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
       }
     }
   ]
   ```

5. Minimal Config Stub
   • Create `dev_config.json` (or inline in code) matching:

   ```json
   {
     "backlog_path": "dev_backlog.json",
     "template_file": "dev_templates.json",
     "src_root": "cadence",
     "ruleset_file": null,
     "repo_dir": ".",
     "record_file": "dev_record.json"
   }
   ```

6. Smoke-Test Orchestrator
   ```bash
   python -m cadence.dev.orchestrator start
   ```
   Expected flow:
   • Executor builds patch (from backlog-diff).  
   • Reviewer passes (static rules).  
   • ShellRunner applies patch.  
   • Pytest turns GREEN.  
   • Commit & archive recorded to `dev_record.json`.

7. Verify Audit Artefacts
   • Open `dev_record.json` → confirm states: build_patch → patch_built → … → archived.  
   • `git log -1` shows commit labeled “[Cadence] bug-fix-add-001 …”.

8. Push to Remote / PR
   • `git remote add origin …` (if needed)  
   • `git push -u origin main`

9. Stretch (optional, if time remains)
   • Run `streamlit run cadence/dev/command_center.py` and walk through the same task via UI.  
   • Add a lint rule to `ruleset.json` (e.g., forbid “print(”) and point config at it to see reviewer fail.

––––––
Once the above is green you have an end-to-end proof that the agentic pipeline can fix a real bug. The next milestone (Story S-MVP-1.2) is to shift from hand-authored diff to LLM-generated diff via `ExecutionAgent`; we’ll tackle that after the smoke test is stable.

In [ ]:
dev_backlog = [
    {
        "id": "bug-fix-add-001",
        "title": "Fix utils.add.add() so that add(2,3)==5",
        "type": "micro",
        "status": "open",
        "created_at": "2025-06-21T00:00:00Z",
        "diff": {
            "file": "cadence/utils/add.py",
            "before": "def add(x: int, y: int) -> int:\\n    return x - 1 + y\\n",
            "after":  "def add(x: int, y: int) -> int:\\n    return x + y\\n"
        }
    }
]

dev_config = {
    "backlog_path": "dev_backlog.json",
    "template_file": "dev_templates.json",
    "src_root": "cadence",
    "ruleset_file": None,
    "repo_dir": ".",
    "record_file": "dev_record.json"
}

21987